In [37]:
import cv2
import nibabel as nib
import numpy as np
from pycimg import CImg
import matplotlib.pyplot as plt


In [38]:
fname = './Images/IMG_0001.nii.gz'
lungs = nib.load(fname).get_fdata()
fname = './BodyMasks/BODYMASK_IMG_0001.nii.gz'
body_mask = nib.load(fname).get_fdata()
max_val = np.max(lungs)

lungs_bin_inv = np.zeros(lungs.T.shape)
for i, dim in enumerate(lungs.T):
    _, lungs_bin_inv[i] = cv2.threshold(dim, -320, max_val, cv2.THRESH_BINARY_INV)

lungs_bin_inv = lungs_bin_inv.T
# print(lungs_bin_inv.shape)
# CImg(lungs_bin_inv).display();
lungs_air_within_body = np.logical_and(lungs_bin_inv, body_mask).astype(np.uint8)
CImg(lungs_air_within_body).display();


In [39]:
lungs_morphed = np.zeros(lungs_air_within_body.T.shape)
for i, dim in enumerate(lungs_air_within_body.T):
    lungs_morphed[i] = cv2.medianBlur(dim, 7)
lungs_morphed = lungs_morphed.T

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
lungs_morphed = cv2.dilate(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.erode(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.erode(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.dilate(lungs_morphed, kernel, None, iterations=2)
CImg(lungs_morphed).display();




In [40]:
lungs_indexed = np.zeros(lungs_morphed.T.shape)
for i, dim in enumerate(lungs_morphed.T):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dim.astype(np.uint8))
    for j in range(num_labels):
        if stats[j][cv2.CC_STAT_AREA] < 500:
            labels[labels == j] = 0

    lungs_indexed[i] = labels

print(num_labels)
for i, dim in enumerate(lungs_indexed):
    _, lungs_indexed[i] = cv2.threshold(dim, 0, 255, cv2.THRESH_BINARY)

lungs_indexed = lungs_indexed.T.astype(np.uint8)
CImg(lungs_indexed).display();



1


In [62]:
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage import filters

lungs_done = np.zeros(lungs_indexed.T.shape)
for i, (segmented, img) in enumerate(zip(lungs_indexed.T, np.logical_not(lungs_bin_inv.T))):
    # distance = ndi.distance_transform_edt(img)
    # coords = peak_local_max(distance, footprint=np.ones((256, 512)), labels=dim)
    # mask = np.zeros(distance.shape, dtype=bool)
    # mask[tuple(coords.T)] = True
    # markers, _ = ndi.label(mask)
    grad = filters.sobel(img)
    marker = cv2.erode(segmented, np.ones((9, 9)))
    _, labels = cv2.connectedComponents(marker)
    labels += 1
    lungs_done[i] = watershed(img, labels, mask=segmented)


lungs_done = lungs_done.T
CImg(lungs_done).display();


In [64]:
lungs_done = watershed(lungs, lungs_indexed, mask=lungs_indexed)
CImg(lungs_done).display();


In [42]:
from skimage import filters

markers = np.zeros(lungs_indexed.T.shape)
for i, dim in enumerate(lungs_indexed.T):
    distance = ndi.distance_transform_edt(dim)
    coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=dim)
    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers[i], _ = ndi.label(mask)

markers = markers.T
CImg(markers).display();